# Business Understanding

## Problem Statement
The large increase in shipping demand has not been matched by an increase in the capabilities of logistics companies.<br>
Delayed delivery can be a risk in many sectors, one of which is retail sales in e-commerce, late delivery will cause the product supply chain to be hampered and reduce the credibility of the retailer. Apart from that, delays by the expedition will also cause buyer disappointment which of course can be detrimental to the retailer.

## Goal
Building a **binary classification** machine learning model that can **predict delays** in logistics/product delivery in e-commerce with **high accuracy**

## Objectives
1. Analyze the data and determine the target feature/binary label (is_late -> (1 or 0), according to the problem statement (delay in delivery)
2. Carry out data processing, to produce data that is clean from noise
3. Carrying out feature engineering, by creating new features to add data patterns which will make it easier for the model to carry out classification (so it is hoped that the accuracy will increase)
4. Select features with high importance using feature importance techniques (Pearson Correlation Matrix, KBest, ChiSquare, and SHAP), to reduce model complexity, computational load, and improve model performance
5. Carry out modeling using several Baseline algorithms (Logistic Regression, SVM, and Decision Tree), as well as advanced algorithms using Ensemble Learning (XGBoost, LGBM, CatBoost, Adaboost, and Random Forest)
6. Evaluate the model with accuracy metrics

# Data Understanding

The data is divided into 2 parts, namely training data and testing data. Each section has 5 types of tables, namely the df_Customers, df_OrderItems, df_Orders, df_Payments, and df_Products tables. <br>
The following is a further explanation of each data:
1. Table df_Customers <br>
This table contains data on customers who make product transactions on ecommerce
2. Table df_OrderItems <br>
This table contains a mapping between orders placed by customers and the table of products purchased
3. Table df_Orders <br>
This table contains data on orders placed by each user
4. df_Payments table <br>
This table contains payments made by each user, containing payment details and transaction value
5. df_Products table <br>
This table contains a list of products sold on ecccomercce and contained in transactions <br>
Let's look at the detailed contents of the table

## Importing Common Libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.set_option('display.max_columns', 99)


In [7]:
import warnings 
warnings.filterwarnings("ignore")

In [10]:
import kagglehub

path = kagglehub.dataset_download("bytadit/ecommerce-order-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/arunekambaram/.cache/kagglehub/datasets/bytadit/ecommerce-order-dataset/versions/1


In [ ]:
tables = ['Orders','Customers','Products','Payments','OrderItems']

## Train data

In [15]:
train_path = "/Users/arunekambaram/Desktop/supplychain-ml/data/Ecommerce Order Dataset/train/df_"
data_list = ['Orders','Customers','Products','Payments','OrderItems']

tr_orders = pd.read_csv(train_path + data_list[0] + '.csv')
tr_custs = pd.read_csv(train_path + data_list[1] + '.csv')
tr_prods = pd.read_csv(train_path + data_list[2] + '.csv')
tr_pays = pd.read_csv(train_path + data_list[3] + '.csv')
tr_items = pd.read_csv(train_path + data_list[4] + '.csv')

In [16]:
tr_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18
...,...,...,...,...,...,...,...
89311,zlcLnXGgAabl,bfJtzkBGksdY,delivered,2018-08-19 17:25:36,2018-08-21 04:35:30,2018-08-30 14:47:46,2018-09-04
89312,IlTAoIkILwrq,4hjsCpj6FC7g,delivered,2017-12-15 11:20:27,2017-12-16 02:37:32,2017-12-20 19:23:40,2018-01-04
89313,uA6oXfftGVmT,DLn56oKHl9bv,delivered,2018-07-24 06:39:14,2018-07-24 11:20:39,2018-07-27 01:41:40,2018-08-01
89314,v3QtU6xlGDJp,auSfRYUQUOIq,delivered,2018-08-11 08:04:22,2018-08-11 08:24:40,2018-08-14 21:43:51,2018-08-17


In [17]:
train_data = [tr_orders, tr_custs, tr_prods, tr_pays, tr_items]
print("The content of training data")
j = 0
for i in train_data:
    print("Table", tables[j])
    print(i.info())
    print(i.head())
    print('\n')
    j+=1

The content of training data
Table Orders
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       89316 non-null  object
 1   customer_id                    89316 non-null  object
 2   order_status                   89316 non-null  object
 3   order_purchase_timestamp       89316 non-null  object
 4   order_approved_at              89307 non-null  object
 5   order_delivered_timestamp      87427 non-null  object
 6   order_estimated_delivery_date  89316 non-null  object
dtypes: object(7)
memory usage: 4.8+ MB
None
       order_id   customer_id order_status order_purchase_timestamp  \
0  Axfy13Hk4PIk  hCT0x9JiGXBQ    delivered      2017-10-22 18:57:54   
1  v6px92oS8cLG  PxA7fv9spyhx    delivered      2018-06-20 21:40:31   
2  Ulpf9skrhjfm  g3nXeJkGI0Qw    delivered      2018-02-16 16

## Test data

In [18]:
test_path = "/Users/arunekambaram/Desktop/supplychain-ml/data/Ecommerce Order Dataset/test/df_"
data_list = ['Orders','Customers','Products','Payments','OrderItems']


ts_orders = pd.read_csv(test_path + data_list[0] + '.csv')
ts_custs = pd.read_csv(test_path + data_list[1] + '.csv')
ts_prods = pd.read_csv(test_path + data_list[2] + '.csv')
ts_pays = pd.read_csv(test_path + data_list[3] + '.csv')
ts_items = pd.read_csv(test_path + data_list[4] + '.csv')

In [19]:
test_data = [ts_orders, ts_custs, ts_prods, ts_pays, ts_items]
print("The content of testing data")
j = 0
for i in test_data:
    print("Table", tables[j])
    print(i.info())
    print(i.head())
    print('\n')
    j+=1

The content of testing data
Table Orders
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38279 entries, 0 to 38278
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   order_id                  38279 non-null  object
 1   customer_id               38279 non-null  object
 2   order_purchase_timestamp  38279 non-null  object
 3   order_approved_at         38272 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB
None
       order_id   customer_id order_purchase_timestamp    order_approved_at
0  u6rPMRAYIGig  I74lXDOfoqsp      2017-11-18 12:29:57  2017-11-18 12:46:08
1  ohY8f4FEbX19  47TuLHF2s7X5      2018-06-02 17:13:12  2018-06-02 20:12:23
2  I28liQek73i2  dQ0dqI8Qwlj8      2018-01-08 11:01:30  2018-01-09 07:24:03
3  bBG1T89mlY8W  iQCmWhNkIczb      2017-03-10 10:24:46  2017-03-10 10:24:46
4  CYxJJSQS8Lbo  Dp2g6JH8tO5Z      2017-12-02 10:04:07  2017-12-05 04:13:30


Table Customers
<class '

### Column Analysis
#### Customers table
1. **customer_id** <br>
ID of each customer who makes a transaction
2. **customer_zip_code_prefix** <br>
prefix of each customer's postal code
3. **customer_city** <br>
the city where the customer lives/the goods will be sent
4. **customer_state** <br>
state/province where the customer lives/items will be shipped

#### Order Items table
1. **product_id** <br>
ID of each product purchased by the customer
2. **seller_id** <br>
ID of the seller/seller for each item in each order
3. **price** <br>
the price of each item sold
4. **shipping_charges** <br>
shipping costs for each item

#### Orders Table
1. **order_id** <br>
ID of each order or orders placed
2. **customer_id** <br>
ID of each customer who makes a transaction
3. **order_status** <br>
status of each order placed
4. **order_purchase_timestamp** <br>
the time at which the purchase is made/initiated by the customer
5. **order_approved_at** <br>
the time when the order is received by the seller
6. **order_delivered_timestamp** <br>
the time when the order has been received by the customer
7. **order_estimated_delivery_date** <br>
the estimated time at which the order is expected to arrive

#### Payments Table
1. **order_id** <br>
ID of each order or orders placed
2. **payment_sequential** <br>
order of payment / payment installments by the customer
3. **payment_type** <br>
type of payment made by the customer
4. **payment_installments** <br>
The number of installments chosen by the customer for each order
5. **payment_value** <br>
payment value for each order, sequentially


#### Products table
1. **product_id** <br>
ID of each product sold
2. **product_category_name** <br>
category name of each product sold
3. **product_weight_g** <br>
weight of each product in grams 
3. **product_length_cm** <br>
Width of each product in cm
4. **product_height_cm** <br>
Height of each product in cm
5. **product_width_cm** <br>
length of each product in cm


### Differences between Training and Testing Data

It turns out there is a difference, that in testing there is only order_id, customer_id, order_purchase_timestamp, and order_approved_at. <br> So, in the training data, the **order_status, order_delivered_timestamp, and order_estimated_delivery_date** columns will be used as **label makers**

## Exploratory Data Analysis (EDA)

### Merging Data

### Train